## 最主要成就：
* 首当其冲的就是，这里实现了一套parse changelog的方法。什么基于etree的解析方法，都tnd不好用，还是简单粗暴的regular expression实用。真是搞得我太tnd不爽了。

Get some necessary information from original code. 

And the code in the following cell comes from Get_Changelog.ipynb

In [1]:
import os, json, md2py
import os, shutil, re, json, tqdm
from lxml import etree

JSON_FILE = r"J:\ModelStoreData\AIHub\2019-06-04\TFModule_Info.json"

def get_element_by_xpath(page_source, xpath, index = 0):
    res = None
    try:
        _ = page_source.xpath(xpath)[index]
        ## eliminate unnecessary chars.
        _ = _.replace("\n", "").strip()
        ## change the strings to lower case. Hopefully it will make the comparison standardized.
        ## be careful about this change. If it is not suitable to the situation, stop it without hesitation.
        _ = _.lower()
        res = ' '.join(_.split())
    except:
        res = None
    finally:
        return res

def parse_html_specially_for_imagenet(html, *args):
    res = None
    try:
        page_source = etree.HTML(html)
        author = get_element_by_xpath(page_source, "//ul/li/text()", args[0])
        author = author.replace(":", "")
        paper = get_element_by_xpath(page_source, "//ul/li/a/text()", args[1])
        paper = paper.replace("\"", "")
        link_to_paper = get_element_by_xpath(page_source, "//ul/li/a/@href", args[2])
        year = get_element_by_xpath(page_source, "//ul/li/text()", args[3])
        year = re.findall(r"(\d{4})", year)[0]
        res = {"author": author, "paper": paper, "link_to_paper":link_to_paper, "year": year}
    except:
        print(f"failed")
    finally:
        return res
    
model_version_logInfo = {}
model_logInfo = {}


def Gather_md_code():
    with open(JSON_FILE, "r") as jf:
        items = json.load(jf)
    counting_hangelog = 0
    counting_ersion = 0
    counting_other = 0
    for item in items:
        name = item["name"]
        version = item["version"]
        info = item["info"]
        link_url = "https://aihub.cloud.google.com/p/products%2F{}/v/{}".format(info[1][1].split("/")[1], version)
        md = info[1][7]
        # toc = md2py.md2py(md)
        # page_source = etree.HTML(str(toc.source))

        ## lets' see whether they have c"hangelog" keywords.
        if "hangelog" in md:
            counting_hangelog += 1
            toc1 = md2py.md2py(md.split("hangelog")[1])
            source_to_write = str(toc1.source)
            remove_references = re.split("<h\d>\Seferences</h\d>", source_to_write)[0]

            model_logInfo[name] = remove_references

            if name not in model_version_logInfo:
                model_version_logInfo[name] = {}  # [[version, str(toc1.source)]]
                model_version_logInfo[name][version] = remove_references  # str(toc1.source)
            else:
                #                 model_version_logInfo[name].append([version, str(toc1.source)])
                model_version_logInfo[name][version] = remove_references  # str(toc1.source)
        #             print(str(toc1.source))
        #             print(name, version, link_url)
        else:
            ## if there is no changelog keyword, let's see "version" keyword.
            ## .. After manually checking out, no version/update/log.. info can be found.
            if "ersion" in md:
                counting_ersion += 1
                # print(md, "\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n")
                # print(name, version, link_url)
            else:
                ## if neither "ersion" nor "hangelog" is available:
                ## .. After manually checking out, no version/update/log.. info can be found.
                counting_other += 1
                # print(name, version, link_url)
                # print(md, "\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n")

    print(counting_hangelog)
    print(counting_ersion)
    print(counting_other)


Gather_md_code()

209
182
40


Get the necessary information from changelog html: 

In [2]:
def parse_html_h4(html, *xpt):
    versions = re.findall(xpt[0], html)
    for i in range(len(versions)):
        versions[i] = versions[i].replace("<h4>", "").replace("</h4>", "")
    return versions

def parse_html_ul_li(html, *xpt):
    uls = re.findall(xpt[0], html)
    list_of_list = []
    for ul in uls:
        lis = re.findall(xpt[1], ul)
        for i in range(len(lis)):
            lis[i] = " ".join(lis[i].replace("<li>", "").replace("</li>", "").split())
        list_of_list.append(lis)
    return list_of_list

Get the last version of each module. 

In [3]:
module_latestVersion = {}
for module in model_version_logInfo:
    for version in model_version_logInfo[module]:
        if module not in module_latestVersion:
            module_latestVersion[module] = version
        else:
            module_latestVersion[module] = module_latestVersion[module] if module_latestVersion[module] > version else version

In [5]:
from pprint import pprint
pprint(model_logInfo)

{'bert_cased_L-12_H-768_A-12': '<h4>Version 1</h4>\n'
                               '<ul>\n'
                               '<li>Initial release.</li>\n'
                               '</ul>\n',
 'bert_cased_L-24_H-1024_A-16': '<h4>Version 1</h4>\n'
                                '<ul>\n'
                                '<li>Initial release.</li>\n'
                                '</ul>\n',
 'bert_chinese_L-12_H-768_A-12': '<h4>Version 1</h4>\n'
                                 '<ul>\n'
                                 '<li>Initial release.</li>\n'
                                 '</ul>\n',
 'bert_multi_cased_L-12_H-768_A-12': '<h4>Version 1</h4>\n'
                                     '<ul>\n'
                                     '<li>Initial release.</li>\n'
                                     '</ul>\n',
 'bert_uncased_L-12_H-768_A-12': '<h4>Version 1</h4>\n'
                                 '<ul>\n'
                                 '<li>Initial release.</li>\n'
               

                                         '<code>tensorflow-hub&gt;=0.2.0</code>.</li>\n'
                                         '</ul>',
 'imagenet-inception_v2-classification': '<h4>Version 1</h4>\n'
                                         '<ul>\n'
                                         '<li>Initial release.</li>\n'
                                         '</ul>\n'
                                         '<h4>Version 3</h4>\n'
                                         '<ul>\n'
                                         '<li>Support for variable input '
                                         'size.</li>\n'
                                         '<li>Fine-tuning: change default '
                                         'batch norm momentum to 0.99 and\n'
                                         '    make it configurable.</li>\n'
                                         '<li>Requires PIP package '
                                         '<code>tensorflow-hub&gt;=0.2.0</code>.</

                                                          '</ul>\n'
                                                          '<h4>Version 3</h4>\n'
                                                          '<ul>\n'
                                                          '<li>Avoids crash in '
                                                          'some TFLite/toco '
                                                          'versions\n'
                                                          '    (<a '
                                                          'href="https://github.com/tensorflow/hub/issues/109">GitHub '
                                                          'issue 109</a>)\n'
                                                          '    by '
                                                          'overestimating '
                                                          'quantization '
                                                          'boundaries 

                                                 '<li>Requires PIP package '
                                                 '<code>tensorflow-hub&gt;=0.2.0</code>.</li>\n'
                                                 '</ul>',
 'imagenet-mobilenet_v1_025_192-feature_vector': '<h4>Version 1</h4>\n'
                                                 '<ul>\n'
                                                 '<li>Initial release.</li>\n'
                                                 '</ul>\n'
                                                 '<h4>Version 3</h4>\n'
                                                 '<ul>\n'
                                                 '<li>Support for variable '
                                                 'input size.</li>\n'
                                                 '<li>Fine-tuning: change '
                                                 'default batch norm momentum '
                                                 'to 0.99 and\n'
  

                                                          '    by '
                                                          'overestimating '
                                                          'quantization '
                                                          'boundaries on input '
                                                          'image by '
                                                          '0.05%.</li>\n'
                                                          '<li>Requires PIP '
                                                          'package '
                                                          '<code>tensorflow-hub&gt;=0.2.0</code>.</li>\n'
                                                          '</ul>',
 'imagenet-mobilenet_v1_025_224-quantops-feature_vector': '<h4>Version 1</h4>\n'
                                                          '<ul>\n'
                                                          '<li>Initial '
                

...

                                                 '<ul>\n'
                                                 '<li>Initial release.</li>\n'
                                                 '</ul>\n'
                                                 '<h4>Version 2</h4>\n'
                                                 '<ul>\n'
                                                 '<li>Fixed broken UPDATE_OPS '
                                                 'for fine-tuning,\n'
                                                 '    <a '
                                                 'href="https://github.com/tensorflow/hub/issues/86">GitHub '
                                                 'issue 86</a>.</li>\n'
                                                 '</ul>\n'
                                                 '<h4>Version 3</h4>\n'
                                                 '<ul>\n'
                                                 '<li>Fine-tuning: change '
                  

                                                 '<li>Fixed broken UPDATE_OPS '
                                                 'for fine-tuning,\n'
                                                 '    <a '
                                                 'href="https://github.com/tensorflow/hub/issues/86">GitHub '
                                                 'issue 86</a>.</li>\n'
                                                 '</ul>\n'
                                                 '<h4>Version 3</h4>\n'
                                                 '<ul>\n'
                                                 '<li>Fine-tuning: change '
                                                 'default batch norm momentum '
                                                 'to 0.99 and\n'
                                                 '    make it '
                                                 'configurable.</li>\n'
                                                 '<li>Requires 

                                                 '</ul>\n'
                                                 '<h4>Version 3</h4>\n'
                                                 '<ul>\n'
                                                 '<li>Fine-tuning: change '
                                                 'default batch norm momentum '
                                                 'to 0.99 and\n'
                                                 '    make it '
                                                 'configurable.</li>\n'
                                                 '<li>Requires PIP package '
                                                 '<code>tensorflow-hub&gt;=0.2.0</code>.</li>\n'
                                                 '</ul>',
 'imagenet-mobilenet_v2_140_224-feature_vector': '<h4>Version 1</h4>\n'
                                                 '<ul>\n'
                                                 '<li>Initial release.</li>\n'
             

 'imagenet-resnet_v1_101-classification': '<h4>Version 1</h4>\n'
                                          '<ul>\n'
                                          '<li>Initial release.</li>\n'
                                          '</ul>\n'
                                          '<h4>Version 3</h4>\n'
                                          '<ul>\n'
                                          '<li>Support for variable input '
                                          'size.</li>\n'
                                          '<li>Fine-tuning: change default '
                                          'batch norm momentum to 0.99 and\n'
                                          '    make it configurable.</li>\n'
                                          '<li>Requires PIP package '
                                          '<code>tensorflow-hub&gt;=0.2.0</code>.</li>\n'
                                          '</ul>',
 'imagenet-resnet_v1_101-feature_vector': '<h4>Version 1</h4>\n'
   

 'universal-sentence-encoder-large': '<h4>Version 1</h4>\n'
                                     '<ul>\n'
                                     '<li>Initial release.</li>\n'
                                     '</ul>\n'
                                     '<h4>Version 2</h4>\n'
                                     '<ul>\n'
                                     '<li>Exposed internal variables as '
                                     'Trainable.</li>\n'
                                     '</ul>\n'
                                     '<h4>Version 3</h4>\n'
                                     '<ul>\n'
                                     '<li>Fixed batch invariant <a '
                                     'href="https://github.com/tensorflow/hub/issues/74">bug</a>. '
                                     'This\n'
                                     '   version was retrained and its '
                                     'embedding space differs from previous '
                        

Group the module by change log. 

And the key of the __log_module__ will be each point of changelog. 

In [6]:
log_module = {}
for module in module_latestVersion:
    version = module_latestVersion[module]
    changelog =  model_version_logInfo[module][version]
    # parse_html(changelog.replace("\n", ""), "//h4/text()")
    # parse_html(changelog.replace("\n", ""), "//ul", "//li/text()")
    versions = parse_html_h4(changelog.replace("\n", ""), "<h4>.*?</h4>")
    changelogs = parse_html_ul_li(changelog.replace("\n", ""), "<ul>.*?</ul>", "<li>.*?</li>")
    for v, logs in zip(versions, changelogs):
        for log in logs:
            if log in log_module:
                log_module[log]["{}@{}".format(module, v)] = -1   #.append((module, v))
            else:
                log_module[log] = {"{}@{}".format(module, v): -1}    # = [(module, v)]
with open("log_module_v1.json", "w") as lm:
    json.dump(log_module, lm, indent= 4, sort_keys= True)